In [1]:
import pandas as pd
import sys, numpy as np
sys.path.append("/afs/cs.stanford.edu/u/anenberg/scr/caffe/python/")
import caffe
import lmdb


# location of the list that was used to create LMDB for training/testing
test_list = '../../data/allFrames/lists/fullpath_shuffle_sampled_100_test_list.txt'

N = 156800 # the number of test examples
num_classes = 101 # the number of classes

# Get the labels from the list

In [2]:
def get_label_from_list(list_filename):
    df = pd.read_csv(list_filename, delimiter= ' ', header = None, names = ['filename', 'class_id'])
    return df

df =  get_label_from_list(test_list)
labels = df.class_id.values

# Extract features using the extract_features binary

In [3]:
!./extract.sh 

E0307 13:36:02.303421 17494 extract_features.cpp:54] Using GPU
E0307 13:36:02.303927 17494 extract_features.cpp:60] Using Device_id=2
E0307 13:36:02.854595 17494 upgrade_proto.cpp:618] Attempting to upgrade input file specified using deprecated V1LayerParameter: ./quick_train_test_100frames.prototxt
E0307 13:36:27.671650 17494 extract_features.cpp:134] Extacting Features
E0307 13:37:09.678227 17494 extract_features.cpp:169] Extracted features of 1000 query images for feature blob fc8_allFrames
E0307 13:37:51.527472 17494 extract_features.cpp:169] Extracted features of 2000 query images for feature blob fc8_allFrames
E0307 13:38:31.553643 17494 extract_features.cpp:169] Extracted features of 3000 query images for feature blob fc8_allFrames
E0307 13:39:08.373400 17494 extract_features.cpp:169] Extracted features of 4000 query images for feature blob fc8_allFrames
E0307 13:39:46.031404 17494 extract_features.cpp:169] Extracted features of 5000 query images for feature blob fc8_allFrames
E

## Open and extract the data from the LMDB

In [196]:
env = lmdb.open('/afs/cs.stanford.edu/u/anenberg/scr/CS231N/examples/allFrames_finetune/extracted/features_100frames/', readonly=True)

predicted_labels = [] # for a particular image, this will be the argmax for the scores
data = np.zeros((N, num_classes)) # collect the scores for each image for each class

with env.begin() as txn:
    with txn.cursor() as cursor:
        for i in range(N):
            key = str(i)
            val = cursor.get(key)
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(val)
            # datum.ListFields()[3][1] contains the scores
            data[i, :] = np.array(list(datum.ListFields()[3][1]))
            predicted_labels.append(np.argmax(np.array(list(datum.ListFields()[3][1]))))


# What was the accuracy?

In [291]:
np.mean(np.array(predicted_labels) == labels)

0.24910714285714286

#Normalize scores to compute average precision for each class

In [293]:
normalized_data = np.exp(data)/sum(np.exp(data))

def compute_average_precision(scores, predicted_labels, true_labels, class_id):
    # given a class_id compute the average precision for that class
    df = pd.DataFrame(zip(scores, predicted_labels, true_labels), columns = ['scores', 'predicted_labels', 'true_labels'])
    df = df.sort('scores', ascending=False)
    df['match'] = (df.true_labels == class_id)
    df['precision'] = np.cumsum(df.match)/np.arange(1, N + 1)
    df['recall'] = np.cumsum(df.match)/np.sum(true_labels == class_id)
    df['delta_recall'] = np.diff(np.append([0], df.recall.values))
    df['rect_area'] = df['precision'] * df['delta_recall']
    # print df
    return sum(df.rect_area)

average_precision = [] # list to hold average_precision

for class_id in range(num_classes):
    ca = compute_average_precision(normalized_data[:, class_id], predicted_labels, labels, class_id)
    average_precision.append(ca)

In [294]:
print 'MAP: %2f'%np.mean(average_precision)

MAP: 0.210921
